In [1]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

2023-09-17 22:51:09.536576: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('data/lome_preprocessed.csv',index_col='date_time',parse_dates=True)


In [3]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
weathercode (wmo code),5337.0,48.012554,1.917312e+01,0.000000,51.000000,53.000000,61.000000,65.000000
temperature_2m_max (°C),5337.0,30.098707,1.716348e+00,24.300000,28.700000,30.500000,31.500000,34.000000
temperature_2m_min (°C),5337.0,25.258160,1.198008e+00,20.200000,24.400000,25.300000,26.200000,28.600000
temperature_2m_mean (°C),5337.0,27.370283,1.327455e+00,23.500000,26.300000,27.600000,28.400000,30.300000
apparent_temperature_max (°C),5337.0,35.727225,2.711631e+00,26.600000,33.700000,36.400000,37.900000,41.200000
apparent_temperature_min (°C),5337.0,29.611561,1.928315e+00,18.200000,28.300000,29.800000,31.100000,34.500000
apparent_temperature_mean (°C),5337.0,31.957692,2.047236e+00,25.000000,30.400000,32.400000,33.600000,36.400000
precipitation_sum (mm),5337.0,2.886996,6.117103e+00,0.000000,0.300000,1.000000,3.500000,126.100000
rain_sum (mm),5337.0,2.886996,6.117103e+00,0.000000,0.300000,1.000000,3.500000,126.100000
precipitation_hours (h),5337.0,4.876897,4.714847e+00,0.000000,1.000000,4.000000,7.000000,24.000000


In [4]:
n = len(df)

# Split 70:20:10 (train:validation:test)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

train_df.shape, val_df.shape, test_df.shape

((3735, 21), (1068, 21), (534, 21))

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_df)

train_df[train_df.columns] = scaler.transform(train_df[train_df.columns])
val_df[val_df.columns] = scaler.transform(val_df[val_df.columns])
test_df[test_df.columns] = scaler.transform(test_df[test_df.columns])

In [6]:
train_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
weathercode (wmo code),3735.0,0.746873,0.287842,0.0,0.784615,0.815385,0.938462,1.0
temperature_2m_max (°C),3735.0,0.596067,0.177465,0.0,0.453608,0.639175,0.731959,1.0
temperature_2m_min (°C),3735.0,0.632298,0.151669,0.0,0.518987,0.632911,0.746835,1.0
temperature_2m_mean (°C),3735.0,0.563497,0.195769,0.0,0.411765,0.588235,0.720588,1.0
apparent_temperature_max (°C),3735.0,0.619690,0.185636,0.0,0.479452,0.664384,0.767123,1.0
apparent_temperature_min (°C),3735.0,0.713041,0.125068,0.0,0.627451,0.725490,0.810458,1.0
apparent_temperature_mean (°C),3735.0,0.630692,0.187714,0.0,0.486239,0.669725,0.779817,1.0
precipitation_sum (mm),3735.0,0.022202,0.045604,0.0,0.003172,0.007930,0.027756,1.0
rain_sum (mm),3735.0,0.022202,0.045604,0.0,0.003172,0.007930,0.027756,1.0
precipitation_hours (h),3735.0,0.196631,0.184617,0.0,0.041667,0.166667,0.291667,1.0


In [ ]:
train_df.to_csv('data/lome_train.csv')
val_df.to_csv('data/lome_val.csv')
test_df.to_csv('data/lome_test.csv')